https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab
```



1.Colab 환경 설정



In [38]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.2
    Uninstalling transformers-4.19.2:
      Successfully uninstalled transformers-4.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-b1y52y7z
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-b1y52y7z
  Using cached transformers-4.19.2-py3-none-any.whl (4.2 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [40]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [41]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [42]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [43]:
#GPU 사용
device = torch.device("cuda:0")

In [44]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


2.데이터셋 전처리

In [45]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import pandas as pd
# chatbot_data = pd.read_excel('/content/drive/MyDrive/챗봇/한국어_단발성_대화_데이터셋.xlsx')
csv_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/restaurant/merge_data.csv')
csv_data.shape

# csv_data = csv_data[:2]
csv_data

,no,skey,yy,signgu,rstrt_se,mnmnu,rstrt_nm,adres,telno,rstrt_mentn_co,kwrd_list,kwrd_list_fq_co
0,0,1,2020,창원시,한식,"낙지전골, 낙지복음",낙지천하,의창구 북면 신촌 본포로32,055-255-0885,6,"맛집, 북면, 배달, 음식, 추천, 낙지, 시골, 음식점, 진짜, 기력, 다음주, ...","7, 5, 4, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1..."
1,0,3,2020,창원시,한식,"추어탕,다슬기탕",가마솥추어탕,의창구 북면 천주로 1019-22 (1층),055-298-3555,10,"추어탕, 북면, 맛집, 추천, 다슬기, 가마솥, 들깨, 밥집, 국물, 포장, 한해,...","14, 12, 9, 9, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1,..."
2,0,5,2020,창원시,한식,"어탕국수,메기매운탕",경호강메기탕,의창구 창이대로85번길 5 (명서동),055-283-7669,3,"창원, 맛집, 마산, 진해, 생각, 어탕, 추천, 횟집, 곱창, 돈까스, 리스트, ...","14, 10, 7, 6, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2,..."
3,0,7,2020,창원시,중식,중화요리,국일성,"의창구 의안로31번길 26 (중동,1층)",055-255-3588,7,"추천, 창원, 근처, 맛집, 시티, 유니, 음식, 중국집, 가족, 계동, 도로, 마...","6, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2..."
4,0,8,2020,창원시,중식,중화요리,금화,의창구 의창대로 450,055-273-1864,2,"금화, 식당, 용동, 창원, 노포, 동남, 중국집, 회관, 가능, 고추, 꽃빵, 네...","7, 5, 4, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...
360,0,1127,2020,합천군,한식,한우모듬,황금나무숯불갈비,합천읍 옥산로 15,055-931-2934,1,"돼지갈비, 양념, 추천, 택배, 포장","1, 1, 1, 1, 1"
361,0,1129,2020,합천군,한식,삼겹살,참숯골,합천읍 옥산로 31,055-932-8592,1,"맛집, 공기밥, 기본, 된장, 삼겹, 서비스, 숙성, 온도, 음료수, 이구, 자동,...","2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
362,0,1131,2020,합천군,한식,삼겹살,부자돼지,합천읍 옥산로 84,055-931-5885,63,"체험, 당첨, 포스팅, 후기, 사항, 반드시, 작성, 돼지국밥, 포함, 방문, 불가...","190, 80, 80, 80, 61, 55, 53, 49, 47, 45, 45, 4..."
363,0,1132,2020,합천군,한식,메기매운탕,약천메기탕,합천읍 충효로 113,055-933-8253,3,"산성, 마을, 바위, 갈림길, 능선, 우산, 합천, 의령, 상투, 이정표, 새우, ...","26, 21, 19, 12, 12, 12, 10, 9, 8, 8, 7, 7, 7, ..."


In [47]:
csv_data['kwrd_all'] = ""
csv_data['rstrt_mentn_co_all'] = 0
for i in range(csv_data['kwrd_list'].count()):
  data = csv_data.iloc[i]
  # kwrd_list_sp = data["kwrd_list"].replace(",","").split()
  kwrd_list_sp = data["kwrd_list"].split()
  kwrd_list_fq_co_sp = data["kwrd_list_fq_co"].replace(",","").split()
  kwrd = ""
  # print(kwrd_list_sp)
  # print(kwrd_list_fq_co_sp)
  for j in range(len(kwrd_list_sp)):
    kwrd += kwrd_list_sp[j] * int(kwrd_list_fq_co_sp[j])
  csv_data.at[i,'kwrd_all'] = kwrd
  csv_data.at[i,'rstrt_mentn_co_all'] = csv_data.at[i,'rstrt_mentn_co'] // 10

csv_data

,no,skey,yy,signgu,rstrt_se,mnmnu,rstrt_nm,adres,telno,rstrt_mentn_co,kwrd_list,kwrd_list_fq_co,kwrd_all,rstrt_mentn_co_all
0,0,1,2020,창원시,한식,"낙지전골, 낙지복음",낙지천하,의창구 북면 신촌 본포로32,055-255-0885,6,"맛집, 북면, 배달, 음식, 추천, 낙지, 시골, 음식점, 진짜, 기력, 다음주, ...","7, 5, 4, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1...","맛집,맛집,맛집,맛집,맛집,맛집,맛집,북면,북면,북면,북면,북면,배달,배달,배달,배...",0
1,0,3,2020,창원시,한식,"추어탕,다슬기탕",가마솥추어탕,의창구 북면 천주로 1019-22 (1층),055-298-3555,10,"추어탕, 북면, 맛집, 추천, 다슬기, 가마솥, 들깨, 밥집, 국물, 포장, 한해,...","14, 12, 9, 9, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1,...","추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어탕,추어...",1
2,0,5,2020,창원시,한식,"어탕국수,메기매운탕",경호강메기탕,의창구 창이대로85번길 5 (명서동),055-283-7669,3,"창원, 맛집, 마산, 진해, 생각, 어탕, 추천, 횟집, 곱창, 돈까스, 리스트, ...","14, 10, 7, 6, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2,...","창원,창원,창원,창원,창원,창원,창원,창원,창원,창원,창원,창원,창원,창원,맛집,맛...",0
3,0,7,2020,창원시,중식,중화요리,국일성,"의창구 의안로31번길 26 (중동,1층)",055-255-3588,7,"추천, 창원, 근처, 맛집, 시티, 유니, 음식, 중국집, 가족, 계동, 도로, 마...","6, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2...","추천,추천,추천,추천,추천,추천,창원,창원,창원,창원,근처,근처,근처,맛집,맛집,맛...",0
4,0,8,2020,창원시,중식,중화요리,금화,의창구 의창대로 450,055-273-1864,2,"금화, 식당, 용동, 창원, 노포, 동남, 중국집, 회관, 가능, 고추, 꽃빵, 네...","7, 5, 4, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1...","금화,금화,금화,금화,금화,금화,금화,식당,식당,식당,식당,식당,용동,용동,용동,용...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,0,1127,2020,합천군,한식,한우모듬,황금나무숯불갈비,합천읍 옥산로 15,055-931-2934,1,"돼지갈비, 양념, 추천, 택배, 포장","1, 1, 1, 1, 1","돼지갈비,양념,추천,택배,포장",0
361,0,1129,2020,합천군,한식,삼겹살,참숯골,합천읍 옥산로 31,055-932-8592,1,"맛집, 공기밥, 기본, 된장, 삼겹, 서비스, 숙성, 온도, 음료수, 이구, 자동,...","2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1","맛집,맛집,공기밥,기본,된장,삼겹,서비스,숙성,온도,음료수,이구,자동,직접,참숯,체...",0
362,0,1131,2020,합천군,한식,삼겹살,부자돼지,합천읍 옥산로 84,055-931-5885,63,"체험, 당첨, 포스팅, 후기, 사항, 반드시, 작성, 돼지국밥, 포함, 방문, 불가...","190, 80, 80, 80, 61, 55, 53, 49, 47, 45, 45, 4...","체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체험,체...",6
363,0,1132,2020,합천군,한식,메기매운탕,약천메기탕,합천읍 충효로 113,055-933-8253,3,"산성, 마을, 바위, 갈림길, 능선, 우산, 합천, 의령, 상투, 이정표, 새우, ...","26, 21, 19, 12, 12, 12, 10, 9, 8, 8, 7, 7, 7, ...","산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산성,산...",0


In [48]:
data_list = []
for q, label in zip(csv_data['kwrd_all'] , csv_data['rstrt_mentn_co_all'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)
# data_list

In [49]:
print(len(data_list))
print(data_list[0])
print(data_list[60])
print(data_list[120])
print(data_list[180])
print(data_list[240])
print(data_list[300])
print(data_list[-1])

365
['맛집,맛집,맛집,맛집,맛집,맛집,맛집,북면,북면,북면,북면,북면,배달,배달,배달,배달,음식,음식,음식,추천,추천,추천,낙지,낙지,시골,시골,음식점,음식점,진짜,진짜,기력,다음주,무동,세명,소개,아부지,요즘,용포탕,이사오,천하,최고', '0']
['추천,추천,추천,추천,추천,추천,추천,추천,추천,추천,맛집,맛집,맛집,맛집,맛집,맛집,맛집,맛집,냉면,냉면,냉면,냉면,냉면,냉면,냉면,마산,마산,마산,마산,마산,마산,소고기,소고기,소고기,소고기,소고기,거리,거리,거리,거리,바로가기,바로가기,바로가기,바로가기,양식,양식,양식,양식,공지,공지,공지,혹시,혹시,혹시,게시판,게시판,고급,고급,드림,드림,메뉴,메뉴,아래,아래,오픈,오픈,일식,일식,정보,정보,창원,창원,코로나코로나', '0']
['불고기,불고기,석쇠,석쇠,분꼭,사진,살수,아시,언니,임각,임신,제보,혹시', '0']
['사건,사건,사건,총기,총기,근처,남강,뭣때문,스톡홀름,시내,식당,코로나', '0']
['수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,수산시장,거제,거제,거제,거제,거제,거제,거제,거제,거제,거제,바다장어,바다장어,바다장어,바다장어,바다장어,바다장어,바다장어,바다장어,바다장어,장어,장어,장어,장어,장어,장어,장어,가상,가상,가상,가상,가상,가상,구매,구매,구매,구매,구매,상태,상태,상태,상태,상태,소매물도,소매물도,소매물도,소매물도,소매물도,정말,정말,정말,정말,정말,후기,후기,후기,후기,후기,고기,고기,고기,고기,이번,이번,이번,이번,주문,주문,주문,주문,최고,최고,최고,최고,추천,추천,추천,추천,캠핑,캠핑,캠핑,캠핑,택일,택일,택일,택일,가시,가시,가시,가족,가족,가족,마무리마무리마무리', '0']
['남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,남해,경남,경남,경남,경남,경남,경남,경남,

3.Train data & Test data

In [50]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [51]:
print(len(dataset_train))
print(len(dataset_test))

273
92


4.KoBERT 입력 데이터로 만들기

In [52]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [53]:
# Setting parameters
# max_len = 64
# batch_size = 64
# warmup_ratio = 0.1
# num_epochs = 5
# max_grad_norm = 1
# log_interval = 200
# learning_rate =  5e-5
max_len = 16
batch_size = 16
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 20
learning_rate =  5e-5

In [54]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [55]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


5.KoBERT 학습모델 만들기

In [56]:
class BERTClassifier(nn.Module):
    # def __init__(self,
    #              bert,
    #              hidden_size = 768,
    #              num_classes=7,   ##클래스 수 조정##
    #              dr_rate=None,
    #              params=None):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):   
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [57]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


6.KoBERT 모델 학습시키기

In [58]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/18 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.337268829345703 train acc 0.125
epoch 1 train acc 0.6666666666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.8576388888888888


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.7194178104400635 train acc 0.8125
epoch 3 train acc 0.8472222222222222


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 3 test acc 0.8576388888888888


7.새로운 문장 테스트

In [63]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            search_lv =  np.argmax(logits)

        print(">> 입력하신 내용은 검색 0 ~ 10 level 중에서 " + str(search_lv) + " level 입니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 남해에 가면 낙지가 많다


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용은 검색 0 ~ 10 level 중에서 0 level 입니다.


하고싶은 말을 입력해주세요 : 낙지
>> 입력하신 내용은 검색 0 ~ 10 level 중에서 0 level 입니다.


하고싶은 말을 입력해주세요 : 낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,낙지,
>> 입력하신 내용은 검색 0 ~ 10 level 중에서 0 level 입니다.


